# Manipulation de données en Python

Nous allons découvrir les capacités de la librairie pandas de Python en chargeant et en manipulant des données sur le très haut débit ainsi que des données géographiques générales de France.

In [ ]:
# imports
import pandas as pnd
import numpy as np
import matplotlib.pyplot as plt

# commande magique pour l'affichage des graphiques
%matplotlib inline

# options d'affichage
from pandas import set_option
set_option("display.max_rows", 16)
plt.style.use('seaborn-notebook')

## Chargement des données

### Données débits internet
Il s'agit des données de couverture sur le très haut débit en France par commune, par technologie (ADSL, câble et fibre optique FttH) et par débit (éligible / 3 / 8 / 30 / 100 Mbit/s).

Les données open data proviennent du portail gouvernemental de la Mission Très Haut Débit accessible à l'adresse http://francethd.fr

In [ ]:
thd = pnd.read_excel("FranceTHD_Open_Data_Observatoire_Juin2015.xlsx",
                            sheetname="Communes",
                            index_col="Code INSEE",
                            header=1,
                            names=["Département", "Commune",
                            "1 Mbit", "3 Mbit", "8 Mbit", "30 Mbit", "100 Mbit",
                            "DSL 1 Mbit", "DSL 3 Mbit", "DSL 8 Mbit", "DSL 30 Mbit", "DSL 100 Mbit",
                            "Câble 1 Mbit", "Câble 3 Mbit", "Câble 8 Mbit", "Câble 30 Mbit", "Câble 100 Mbit",
                            "Fibre 1 Mbit", "Fibre 3 Mbit", "Fibre 8 Mbit", "Fibre 30 Mbit", "Fibre 100 Mbit"])
thd

### Données INSEE

In [ ]:
geo = pnd.read_csv("correspondance-code-insee-code-postal.csv",
                          sep=';',
                          usecols=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                          index_col="Code INSEE")
geo

In [ ]:
# décompte des statuts des communes
geo["Statut"].value_counts()

In [ ]:
# décompte des noms de commune les plus fréquents
thd["Commune"].value_counts().head(10)

In [ ]:
# débits toutes technos des communes dénommées "Sainte-Colombe"
thd[thd["Commune"] == "Sainte-Colombe"]

In [ ]:
thd.info()

## Test sur les données

In [ ]:
# on vérifie que dans chaque technologie le % est supérieur ou égal au débit supérieur 
for i in range(2, len(thd.columns), 5):
    for j in range(i, i + 4):
        test = (thd.iloc[:, j] >= thd.iloc[:, j + 1]).all()
        print(thd.columns[j], ">=", thd.columns[j + 1], ":", test)

In [ ]:
# on vérifie que que le % global est supérieur ou égal au % dans chaque technologie à débit identique 
for i in range(2, 7):
    for j in range(i + 5, len(thd.columns), 5):
        test = (thd.iloc[:, i] >= thd.iloc[:, j]).all()
        print(thd.columns[i], ">=", thd.columns[j], ":", test)

## Correction des données

In [ ]:
# correction des données
# cette instruction considère implicitement que les données de la colonne "Commune" sont des chaines de caractères renvoie une erreur
thd[thd["Commune"].str.startswith("Saint")]

In [ ]:
# cette instruction est correcte
thd["Commune"].str.startswith("Saint")

In [ ]:
# décompte des valeurs y compris NaN
(thd["Commune"].str.startswith("Saint")).value_counts(dropna=False)

In [ ]:
# quelles sont les valeurs Nan
thd[thd["Commune"].str.startswith("Saint").isnull()]

In [ ]:
# Ouvrir le fichier Excel

In [ ]:
# correction des données
thd.loc["08165", "Commune"] = "FAUX"
thd.loc["24177", "Commune"] = "FAUX"
thd.loc[["08165", "24177"]]

In [ ]:
thd[thd["Commune"].str.startswith("Saint")]

## Conversion de type

In [ ]:
# conversion de la colonne Statut en catégorie
geo["Statut"] = geo["Statut"].astype('category',
                                     categories=["Commune simple", "Chef-lieu canton", "Sous-préfecture",
                                                "Préfecture", "Préfecture de région", "Capitale d'état"],
                                     ordered=True)
geo.info()

## Ajout de données

In [ ]:
# on ajoute la colonne "CP Ville"
geo["CP Ville"] = geo["Code Postal"] + " " + geo["Commune"]  # ou bien geo["CP Ville"] = geo["Code Postal"].str.cat(geo["Commune"], sep=' ')
geo.head()

In [ ]:
# la colonne "geo_point_2d" est constituée de chaînes de caractères
geo["geo_point_2d"]

In [ ]:
# application de la méthode split()
x = geo.loc["31080", "geo_point_2d"].split(', ')
x

In [ ]:
# on calcule  la latitude et la longitude et on ajoute les colones
geo["Latitude"] = geo["geo_point_2d"].apply(lambda x: float(x.split(', ')[0]))
geo["Longitude"] = geo["geo_point_2d"].apply(lambda x: float(x.split(', ')[1]))
geo.head()

## Recherche dans les données

In [ ]:
# fonction recherche de ville
def ville(latitude, longitude):
    ecart_latitude = geo["Latitude"] - latitude
    ecart_longitude = geo["Longitude"] - longitude
    dist = ecart_latitude.combine(ecart_longitude, lambda x, y: x * x + y * y)
    return geo["CP Ville"].loc[dist.idxmin()]

In [ ]:
# on applique la fonction à une coordonnée tirée au hasard
a, b = 41.5, 51.1  # latitude min et max de la France métropolitaine
latitude = (b - a) * np.random.random_sample() + a
a, b = -5.1, 9.5  # longitude min et max de la France métropolitaine
longitude = (b - a) * np.random.random_sample() + a
print(latitude, longitude)
ville(latitude, longitude)

In [ ]:
# conversion degrés,minutes,secondes => décimal
def dms2dec(d, m, s):
    return d + (m / 60) + (s / 3600)

In [ ]:
# à partir de coordonnées GPS précises
ville(dms2dec(48, 50, 0),dms2dec(2, 15, 0))

## Visualisation

In [ ]:
## scatter plot
geo.plot(kind='scatter', x="Longitude", y="Latitude");

In [ ]:
# condition pour n'afficher que la France métropolitaine
# avec la latitude
metro = geo[geo["Latitude"] > 40]
# avec la longitude
# metro = geo[np.abs(geo["Longitude"]) < 10]
# avec le code postal
# metro = geo[geo["Code Postal"] < "96000"]
metro.plot(kind='scatter', x="Longitude", y="Latitude", title="Projection de Mercator");

Les cartes sont déformées car la projection cartésienne d'une sphère à tendance à élargir les distances horizontales (longitudinales) qui sont éloignées de l'équateur (à latitude élevée).

On peut corriger le problème en affichant les valeurs en km et non plus en degrés. Le rayon équatorial vaut $R_{équatorial} = 6378,137  km$ et le rayon polaire vaut $R_{polaire} = 6356,752  km$.

La distance entre 2 points sur l'équateur et séparés par un angle longitudinal $\lambda$ vaut $\lambda \times R_{équatorial}$ où $\lambda$ est exprimé en radiants ($360° = 2 \times \pi$).

La distance entre 2 points sur un même parallèle de latitude $\phi$ et séparés par un angle longitudinal $\lambda$ vaut $\lambda \times R_{équatorial} \times \cos(\phi)$.

La distance entre 2 points sur un même méridien et séparés par un angle latitudinal $\phi$ vaut $\phi \times R_{polaire}$.


On utilise la fonction de conversion des degrés en radiants *np.deg2rad()*.

N.B. : Cette méthode demeure une approximation due à la forme non sphérique de la Terre.

In [ ]:
# données terrestres
rayon_equatorial = 6378.137
rayon_polaire = 6356.752

# conversions
latitude = np.deg2rad(metro["Latitude"])
longitude = np.deg2rad(metro["Longitude"])

# on crée une figure 1 x 2
fig = plt.figure(figsize=(15, 5))

ax1 = fig.add_subplot(121)
ax1.scatter(metro["Longitude"], metro["Latitude"])
ax1.set_title("Projection de Mercator")

ax2 = fig.add_subplot(122)
ax2.scatter(longitude * rayon_equatorial * np.cos(latitude), latitude * rayon_polaire)
ax2.set_title("Projection cylindrique équidistante");

## Jointure entre DataFrame

In [ ]:
# jointure
alldata = thd.join(geo, lsuffix="_")
alldata

In [ ]:
# on pivote sur les valeurs de la colonne "Statut"
pnd.pivot_table(alldata, values=["1 Mbit", "3 Mbit", "8 Mbit", "30 Mbit", "100 Mbit"], columns=["Statut"])

In [ ]:
# colonnes suplémentaires avec les 4 scores
alldata["Score THD"] = alldata.loc[:, "1 Mbit":"100 Mbit"].mean(axis=1)
alldata["Score DSL"] = alldata.loc[:, "DSL 1 Mbit":"DSL 100 Mbit"].mean(axis=1)
alldata["Score Câble"] = alldata.loc[:, "Câble 1 Mbit":"Câble 100 Mbit"].mean(axis=1)
alldata["Score Fibre"] = alldata.loc[:, "Fibre 1 Mbit":"Fibre 100 Mbit"].mean(axis=1)
alldata.head()

In [ ]:
# on crée une figure 2 x 2
def display(selection):
    fig = plt.figure(figsize=(14, 12))
    ax1 = fig.add_subplot(221)
    sel=selection.sort_values(by="Score THD")
    ax1.scatter(x=sel["Longitude"],
                y=sel["Latitude"],
                c=1 - sel["Score THD"],
                cmap=plt.cm.Spectral,
                edgecolors='none')
    ax1.set_title("Toutes technos")
    ax2 = fig.add_subplot(222)
    sel=selection.sort_values(by="Score DSL")
    ax2.scatter(x=sel["Longitude"],
                y=sel["Latitude"],
                c=1 - sel["Score DSL"],
                cmap=plt.cm.Spectral,
                edgecolors='none')
    ax2.set_title("DSL")
    ax3 = fig.add_subplot(223)
    sel=selection.sort_values(by="Score Câble")
    ax3.scatter(x=sel["Longitude"],
                y=sel["Latitude"],
                c=1 - sel["Score Câble"],
                cmap=plt.cm.Spectral,
                edgecolors='none')
    ax3.set_title("Câble")
    ax4 = fig.add_subplot(224)
    sel=selection.sort_values(by="Score Fibre")
    ax4.scatter(x=sel["Longitude"],
                y=sel["Latitude"],
                c=1 - sel["Score Fibre"],
                cmap=plt.cm.Spectral,
                edgecolors='none');
    ax4.set_title("Fibre")

In [ ]:
display(alldata[alldata["Latitude"]>40])

## Transformation des données

In [ ]:
thd.head()

In [ ]:
thd.reset_index(inplace=True)
thd.set_index(["Code INSEE", "Département", "Commune"], inplace=True)
thd.head()

In [ ]:
# on crée des colonnes hiérarchiques que l'on nomme également
thd.columns = [["Tout"] * 5 + ["DSL"] * 5 + ["Câble"] * 5 + ["Fibre"] * 5,
 ["1 Mbit", "3 Mbit", "8 Mbit", "30 Mbit", "100 Mbit"] * 4]
thd.columns.names = ["Techno", "Débit"]  # on nomme les 2 index"
thd.head()

In [ ]:
thd[('DSL', '3 Mbit')]

In [ ]:
# la méthode stack() fait passer le niveau de colonne le plus bas vers le niveau d'index le plus bas
var = thd.stack()
var

In [ ]:
# la méthode stack() fait passer le niveau de colonne le plus bas vers le niveau d'index le plus bas
var = thd.stack().stack()
var

In [ ]:
data = thd.stack().stack().reset_index()
data.columns=["Code INSEE", "Département", "Commune", "Techno", "Débit", "Valeur"]
data

### Données technos mobiles

In [ ]:
mob = pnd.read_csv("couverture-2g-3g-4g-en-france-par-operateur-juillet-2015.csv", sep=";")
mob.head()

In [ ]:
new_index = list(mob.columns.values[0:6]) + [mob.columns.values[-1]]
mob2 = mob.set_index(new_index)
mob2.head()

In [ ]:
mob2.columns = [["Population"] * 15 + ["Surfacique"] * 15,
                   ["4G"] * 5 + ["3G"] * 5 + ["2G"] * 5 + ["4G"] * 5 + ["3G"] * 5 + ["2G"] * 5,
                    ["Orange", "Bouygues", "SFR", "Free", "Opérateur"] * 6] 
mob2.columns.names = ["Type", "Techno", "Opérateur"]
mob2.head()

In [ ]:
# noms des colonnes hiérarchiques
current_columns = mob2.columns.names
# noms des colonnes de la table finale
new_columns = mob2.index.names + current_columns + ["Value"]
# empilement des colonnes hiérarchiques vers l'index puis annulation de l'index
mob2 = mob2.stack(current_columns).reset_index()
mob2.columns = new_columns
mob2.head()

In [ ]:
mob2[(mob2["NOM COMMUNE"] == "ROUEN") & (mob2["Techno"] == "4G") & (mob2["Opérateur"] == "Orange")]

In [ ]:
mob2[(mob2["NOM COMMUNE"] == "ROUEN") & (mob2["Techno"] == "4G") & (mob2["Type"] == "Population")]

## Affichage des données

In [ ]:
# on calcule  la latitude et la longitude et on ajoute les colones
mob["Latitude"] = mob["coordonnees"].apply(lambda x: float(x.split(', ')[0]))
mob["Longitude"] = mob["coordonnees"].apply(lambda x: float(x.split(', ')[1]))
mob.head()

In [ ]:
# recherche du meilleur opérateur 4G
def best_4G(row):
    m = row["Orange France Couverture surfacique 4G":"Free Mobile Couverture surfacique 4G"].max()
    # 0 pas de couverture, 1 = Orange, 2 = Bouygues, 3 = SFR, 4 = Free
    if m == 0.0:
        return 0
    elif row["Orange France Couverture surfacique 4G"] == m:
        return 1
    elif row["Bouygues Telecom Couverture surfacique 4G"] == m:
        return 2
    elif row["SFR Couverture surfacique 4G"] == m:
        return 3
    else:
        return 4

In [ ]:
mob.apply(best_4G, axis=1).value_counts()

In [ ]:
mob["Best 4G"] = mob.apply(best_4G, axis=1)
mob.head()

In [ ]:
var = mob.sort_values(by="Best 4G")  # la carte dépend fortement de l'ordre d'affichage
colors = ['w', 'y', 'r', 'c', 'k']  # White, Yellow, Red, Cyan, Black
plt.scatter(x=var["Longitude"],
    y=var["Latitude"],
    c=var["Best 4G"].apply(lambda x: colors[x]),
    edgecolors='none');

In [ ]:
var = mob[mob["Best 4G"] != 0]
var = var.sort_values(by="Best 4G", ascending=False)  # la carte dépend fortement de l'ordre d'affichage
colors = ['w', 'y', 'r', 'c', 'k']  # White, Yellow, Red, Cyan, Black
plt.scatter(x=var["Longitude"],
    y=var["Latitude"],
    c=var["Best 4G"].apply(lambda x: colors[x]),
    edgecolors='none');